### Machine Learning Testing - Supply and Demand 
(with our old data) by finding a relationship between our supply and demand data
# ************************************************************************
# Preprocessing


In [148]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error
import pandas as pd
#import tensorflow as tf

In [149]:
# Import and read the csv's
##supply data
key_data = pd.read_csv("../data/old/00-key.csv")
sku_data = pd.read_csv("../data/old/01-SKU.csv")
dist_data = pd.read_csv("../data/old/02-DIST.csv")

##demand data
posa_data = pd.read_csv("../data/old/03-POS-A.csv")
posb_data = pd.read_csv("../data/old/03-POS-B.csv")
sect_data = pd.read_csv("../data/old/04-SECT.csv")

matrix_data = pd.read_csv("../data/old/05-MATRIX.csv")

In [150]:
#display table (CHECKING IF IT RAN CORRECTLY)
#dist_data.head()
posa_data.head()

,BLDG_ID,SqFtCd,SqFt_est,POS_Landuse,sectorid,POS_StateClass,POS_StateClassCd,center_x,center_y
0,317,UNDER 5000SF,1777.219873,NaN,1,RESIDENTIAL,A1,-95.3526,29.76921
1,630,UNDER 5000SF,2179.123515,NaN,1,RESIDENTIAL,A1,-95.3526,29.76901
2,1132,UNDER 5000SF,2127.308579,NaN,1,RESIDENTIAL,A1,-95.3525,29.76926
3,1277,UNDER 5000SF,4427.082666,NaN,1,RESIDENTIAL,A1,-95.3545,29.76754
4,1328,UNDER 5000SF,1467.869286,NaN,1,RESIDENTIAL,A1,-95.3526,29.76871


In [151]:
# Fill NaN with 0 (when running the traininng it will have less issues)
key_data_df = key_data.fillna(0) 
sku_data_df = sku_data.fillna(0) 
dist_data_df = dist_data.fillna(0) 
posa_data_df = posa_data.fillna(0) 
posb_data_df = posb_data.fillna(0) 
sect_data_df = sect_data.fillna(0) 
matrix_data_df = matrix_data.fillna(0) 



In [152]:
# Merge the key and sku dataframes for supply df
combined_supply_df = pd.merge(key_data_df, sku_data_df, on='schema_id')
combined_supply_df.head()

,schema_id,name_desc,avg_stock,avg_price_x,avg_mktshare,SKU_ID,sku_name,avg_price_y,supply_desc,distribution,production
0,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,food packaging,commercial,unfermented
1,S01,Soybeans,802075,0.22,176456,1001021,edemame,5.26,food packaging,commercial,unfermented
2,S02,Soybean meal,683667,0.19,129897,1001018,tofu,2.00,food manufacturing,commercial,unfermented
3,S02,Soybean meal,683667,0.19,129897,1001019,soymilk,3.49,food manufacturing,commercial,unfermented
4,S02,Soybean meal,683667,0.19,129897,1001022,tempeh,7.37,food manufacturing,commercial,fermented


In [153]:
# Merge distributors with all of supply df
combined_supply_df = pd.merge(combined_supply_df, dist_data_df, on='schema_id')
combined_supply_df.head()

,schema_id,name_desc,avg_stock,avg_price_x,avg_mktshare,SKU_ID,sku_name,avg_price_y,supply_desc_x,distribution,production,CUST_ID,CUST__NAME,supply_desc_y,CUST_ADDRESS
0,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,food packaging,commercial,unfermented,2001005,Anu Resources,food packaging,"11757 Katy Freeway, Suite 1300, Houston 77079 ..."
1,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,food packaging,commercial,unfermented,2001006,Al Hakeem Co,food packaging,9318 Lynchester Dr Houston 77083 Texas United ...
2,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,food packaging,commercial,unfermented,2001007,Lien Hoa B. Inc,food packaging,7611 Summer Glen Ln Houston 77072 United States
3,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,food packaging,commercial,unfermented,2001011,Cronusweb Corporation,food packaging,2425 West Loop South Suite 200 Houston 77027 T...
4,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,food packaging,commercial,unfermented,2001014,Kavantures LLC,food packaging,3032 South Camino Lagos Grand Prairie 75054 Te...


In [154]:
# Merge the POS A and POS B dataframes for demand ds
combined_demand_df = pd.merge(posa_data_df, posb_data_df, on='BLDG_ID')
combined_demand_df.head()

,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClass,POS_StateClassCd,center_x,center_y,POS_Contact,...,POS_NAICS,POS_North American Industry Classification,POS_TYPE_focus,POS_Revenue,Owner_Ethnicity,Owner_Type,Owner_Established,Owner_CreditScore,SqFtCd_y,SqFt_range
0,317,UNDER 5000SF,1777.219873,0,1,RESIDENTIAL,A1,-95.3526,29.76921,ABRAM AYALA,...,23822020,PLUMBING HTG & AIR-CONDITIONING CONTRACTORS,0,"LESS THAN $500,000",HISPANIC,FIRM,2019,C+,2.0,"1,500 - 2,499"
1,111,0,5530.625253,0,0,RESIDENTIAL,A1,-95.3466,29.76002,0,...,54161830,OTHER MANAGEMENT CONSULTING SERVICES,0,"LESS THAN $500,000",0,FIRM,2013,C+,2.0,"1,500 - 2,499"
2,169,UNDER 5000SF,1024.561641,0,0,RESIDENTIAL,A1,-95.3556,29.74381,KENNETH CASTANEDA,...,99999004,UNCLASSIFIED ESTABLISHMENTS,0,0,HISPANIC,FIRM,2017,U,1.0,"1 - 1,499"
3,240,0,5479.600115,0,0,RESIDENTIAL,A1,-95.3776,29.75606,0,...,42472014,OTHER PETROLEUM MERCHANT WHOLESALE,0,$100-500 MILLION,0,FIRM,2020,B+,5.0,"10,000 - 19,999"
4,301,0,5431.591758,0,0,RESIDENTIAL,A1,-95.3465,29.76042,NOEVA VYARA,...,99999004,UNCLASSIFIED ESTABLISHMENTS,0,0,UNKNOWN,FIRM,2020,U,1.0,"1 - 1,499"


In [155]:
# Merge sect with all of demand df so we have it on the data what sector id name they are with 
combined_demand_df = pd.merge(combined_demand_df, sect_data_df, on='sectorid')
combined_demand_df.head()

,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClass,POS_StateClassCd,center_x,center_y,POS_Contact,...,POS_North American Industry Classification,POS_TYPE_focus,POS_Revenue,Owner_Ethnicity,Owner_Type,Owner_Established,Owner_CreditScore,SqFtCd_y,SqFt_range,sectorid_name
0,317,UNDER 5000SF,1777.219873,0,1,RESIDENTIAL,A1,-95.3526,29.76921,ABRAM AYALA,...,PLUMBING HTG & AIR-CONDITIONING CONTRACTORS,0,"LESS THAN $500,000",HISPANIC,FIRM,2019,C+,2.0,"1,500 - 2,499",Buffalo Bayou Bend
1,188,0,36231.895740,MULTIFAMILY,1,RESIDENTIAL,B1,-95.3539,29.76787,CARLA BISONG,...,ART DEALERS,0,"LESS THAN $500,000",UNKNOWN,FIRM,2013,C,2.0,"1,500 - 2,499",Buffalo Bayou Bend
2,188,0,36231.895740,MULTIFAMILY,1,RESIDENTIAL,B1,-95.3539,29.76787,0,...,FREIGHT TRANSPORTATION ARRANGEMENT,0,$1-2.5 MILLION,0,FIRM,2018,C+,5.0,"10,000 - 19,999",Buffalo Bayou Bend
3,188,0,36231.895740,MULTIFAMILY,1,RESIDENTIAL,B1,-95.3539,29.76787,THEODORE REX,...,FULL-SERVICE RESTAURANTS,RESTAURANTS,"LESS THAN $500,000",ENGLISH,FIRM,2017,C+,3.0,"2,500 - 4,999",Buffalo Bayou Bend
4,188,0,36231.895740,MULTIFAMILY,1,RESIDENTIAL,B1,-95.3539,29.76787,NEBYOU BEDRU,...,OFFICES OF OTHER HOLDING COMPANIES,0,$2.5-5 MILLION,UNKNOWN,FIRM,2018,B,5.0,"10,000 - 19,999",Buffalo Bayou Bend


In [156]:
# Describe the supply new df
combined_supply_df.describe()

,avg_stock,avg_price_x,avg_mktshare,SKU_ID,avg_price_y,CUST_ID
count,227.000000,227.000000,227.000000,2.270000e+02,227.000000,2.270000e+02
mean,239575.722467,3.079824,72552.383260,1.001013e+06,12.115242,2.001022e+06
std,326686.508202,2.997312,48023.638918,7.956962e+00,20.023863,1.185245e+01
min,3185.000000,0.190000,37213.000000,1.001000e+06,0.670000,2.001000e+06
25%,5390.000000,0.190000,40519.000000,1.001005e+06,4.490000,2.001012e+06
50%,15925.000000,2.540000,40519.000000,1.001014e+06,6.320000,2.001023e+06
75%,683667.000000,6.900000,129897.000000,1.001019e+06,7.370000,2.001029e+06
max,802075.000000,13.660000,176456.000000,1.001026e+06,90.000000,2.001043e+06


In [157]:
# Describe the demand new df
combined_demand_df.describe()

,BLDG_ID,SqFt_est,sectorid,center_x,center_y,Latitude,Longitude,POS_SELSIC,POS_NAICS,Owner_Established,SqFtCd_y
count,4517.000000,4517.000000,4517.000000,4517.000000,4517.000000,4517.000000,4517.000000,4517.000000,4.517000e+03,4517.000000,4517.000000
mean,709.367943,48097.425467,6.816250,-95.364904,29.757020,29.757019,-95.364870,711417.136152,6.078012e+07,2011.384547,3.774408
std,363.931589,57873.994040,3.429522,0.004850,0.005024,0.005038,0.004838,210449.528384,2.003375e+07,8.938713,2.443386
min,102.000000,1024.561641,0.000000,-95.378700,29.738700,29.738700,-95.378700,19101.000000,1.119980e+07,1984.000000,0.000000
25%,442.000000,25272.309730,5.000000,-95.367100,29.755370,29.755500,-95.367000,602103.000000,5.239300e+07,2007.000000,2.000000
50%,658.000000,40819.076550,8.000000,-95.364800,29.757870,29.757900,-95.364700,762902.000000,5.411990e+07,2015.000000,4.000000
75%,1042.000000,60655.105350,9.000000,-95.362100,29.760270,29.760300,-95.362200,811103.000000,7.225112e+07,2018.000000,6.000000
max,1410.000000,806967.910900,11.000000,-95.346500,29.772990,29.772900,-95.346600,999977.000000,9.999902e+07,2022.000000,8.000000


In [158]:
# look at schema_id value count for binning
print(combined_supply_df['schema_id'].value_counts())

B01    90
S02    60
B02    56
S01    14
B03     5
S03     2
Name: schema_id, dtype: int64


In [159]:
# look at bldg_id value count for binning
print(combined_demand_df['BLDG_ID'].value_counts())

658     214
1289    163
610     159
468     141
761     140
       ... 
1156      1
1307      1
554       1
640       1
317       1
Name: BLDG_ID, Length: 368, dtype: int64


In [160]:
# Convert categorical data(selected columns) to numeric in supply df (using get dummies)
supply_cols = ['supply_desc_x','supply_desc_y','distribution','production']
supply_dummies = pd.get_dummies(combined_supply_df, columns=supply_cols)
supply_dummies.head()

,schema_id,name_desc,avg_stock,avg_price_x,avg_mktshare,SKU_ID,sku_name,avg_price_y,CUST_ID,CUST__NAME,CUST_ADDRESS,supply_desc_x_food manufacturing,supply_desc_x_food packaging,supply_desc_y_food manufacturing,supply_desc_y_food packaging,distribution_commercial,distribution_wholesale,production_fermented,production_processed,production_unfermented
0,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,2001005,Anu Resources,"11757 Katy Freeway, Suite 1300, Houston 77079 ...",0,1,0,1,1,0,0,0,1
1,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,2001006,Al Hakeem Co,9318 Lynchester Dr Houston 77083 Texas United ...,0,1,0,1,1,0,0,0,1
2,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,2001007,Lien Hoa B. Inc,7611 Summer Glen Ln Houston 77072 United States,0,1,0,1,1,0,0,0,1
3,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,2001011,Cronusweb Corporation,2425 West Loop South Suite 200 Houston 77027 T...,0,1,0,1,1,0,0,0,1
4,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,2001014,Kavantures LLC,3032 South Camino Lagos Grand Prairie 75054 Te...,0,1,0,1,1,0,0,0,1


In [161]:
# Convert categorical data(selected columns) to numeric in demand df (using get dummies)
dem_cols = ['POS_StateClass','POS_North American Industry Classification','POS_Revenue']
demand_dummies =pd.get_dummies(combined_demand_df, columns=dem_cols)
demand_dummies.head()


,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClassCd,center_x,center_y,POS_Contact,POS_Name,...,POS_Revenue_$1-2.5 MILLION,POS_Revenue_$10-20 MILLION,POS_Revenue_$100-500 MILLION,POS_Revenue_$2.5-5 MILLION,POS_Revenue_$20-50 MILLION,POS_Revenue_$5-10 MILLION,POS_Revenue_$50-100 MILLION,"POS_Revenue_$500,000-$1 MILLION","POS_Revenue_LESS THAN $500,000",POS_Revenue_OVER $1 BILLION
0,317,UNDER 5000SF,1777.219873,0,1,A1,-95.3526,29.76921,ABRAM AYALA,ACE COMFORT AIR COND & HTG,...,0,0,0,0,0,0,0,0,1,0
1,188,0,36231.895740,MULTIFAMILY,1,B1,-95.3539,29.76787,CARLA BISONG,BISONG ART GALLERY,...,0,0,0,0,0,0,0,0,1,0
2,188,0,36231.895740,MULTIFAMILY,1,B1,-95.3539,29.76787,0,IRMA SHIPPING,...,1,0,0,0,0,0,0,0,0,0
3,188,0,36231.895740,MULTIFAMILY,1,B1,-95.3539,29.76787,THEODORE REX,THEODORE REX,...,0,0,0,0,0,0,0,0,1,0
4,188,0,36231.895740,MULTIFAMILY,1,B1,-95.3539,29.76787,NEBYOU BEDRU,RAH HOLDINGS LLC,...,0,0,0,1,0,0,0,0,0,0


In [168]:
# Grabbing the matrix csv file 
matrix_data_df

,POS_TYPE_focus,S01,S02,S03,B01,B02,B03,Total
0,BAR,0.10,0.20,0.40,0.05,0.65,0.10,1.50
1,BEVERAGE STORES,0.00,0.00,0.10,0.00,0.00,0.00,0.10
2,BREWERY,0.00,0.00,0.10,0.00,0.00,0.05,0.15
3,CATERERS,0.25,0.25,0.25,0.25,0.25,0.25,1.50
4,CHAIN RESTAURANTS,0.00,0.10,0.40,0.65,0.00,0.00,1.15
5,ENTERTAINMENT,0.00,0.00,0.40,0.25,0.00,0.00,0.65
6,FOOD STORES,0.10,0.10,0.15,0.10,0.10,0.10,0.65
7,GENERAL STORES,0.10,0.10,0.10,0.10,0.00,0.10,0.50
8,HOTELS,0.10,0.85,0.10,0.25,0.10,0.10,1.50
9,PHARMACY STORES,0.00,0.00,0.05,0.00,0.00,0.05,0.10


In [169]:
# Convert the matrix(wide) df into a longer form
matrix_df = matrix_data_df.melt(id_vars=['POS_TYPE_focus'], var_name='schema_id', value_name='correlation')


In [170]:
# Create a merged demand df with matrix_df so it keeps only data from demand df (connects it together to build our model)
combined_demand_df = pd.merge(demand_dummies, matrix_df, on=['POS_TYPE_focus'], how='left')
combined_demand_df.head()

,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClassCd,center_x,center_y,POS_Contact,POS_Name,...,POS_Revenue_$100-500 MILLION,POS_Revenue_$2.5-5 MILLION,POS_Revenue_$20-50 MILLION,POS_Revenue_$5-10 MILLION,POS_Revenue_$50-100 MILLION,"POS_Revenue_$500,000-$1 MILLION","POS_Revenue_LESS THAN $500,000",POS_Revenue_OVER $1 BILLION,schema_id,correlation
0,317,UNDER 5000SF,1777.219873,0,1,A1,-95.3526,29.76921,ABRAM AYALA,ACE COMFORT AIR COND & HTG,...,0,0,0,0,0,0,1,0,NaN,NaN
1,188,0,36231.895740,MULTIFAMILY,1,B1,-95.3539,29.76787,CARLA BISONG,BISONG ART GALLERY,...,0,0,0,0,0,0,1,0,NaN,NaN
2,188,0,36231.895740,MULTIFAMILY,1,B1,-95.3539,29.76787,0,IRMA SHIPPING,...,0,0,0,0,0,0,0,0,NaN,NaN
3,188,0,36231.895740,MULTIFAMILY,1,B1,-95.3539,29.76787,THEODORE REX,THEODORE REX,...,0,0,0,0,0,0,1,0,S01,0.25
4,188,0,36231.895740,MULTIFAMILY,1,B1,-95.3539,29.76787,THEODORE REX,THEODORE REX,...,0,0,0,0,0,0,1,0,S02,0.25


In [174]:
# Get volume for sku
combined_demand_df['volume_sku'] = (
    (combined_demand_df['SqFt_est'] * combined_demand_df['correlation'])
    / (
        combined_demand_df['POS_Revenue_$1-2.5 MILLION'] * 100
        + combined_demand_df['POS_Revenue_$2.5-5 MILLION'] * 150
        + combined_demand_df['POS_Revenue_$5-10 MILLION'] * 200
        + combined_demand_df['POS_Revenue_$500,000-$1 MILLION'] * 50
        + combined_demand_df['POS_Revenue_LESS THAN $500,000'] * 25
    )
)
combined_demand_df.head()

,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClassCd,center_x,center_y,POS_Contact,POS_Name,...,POS_Revenue_$2.5-5 MILLION,POS_Revenue_$20-50 MILLION,POS_Revenue_$5-10 MILLION,POS_Revenue_$50-100 MILLION,"POS_Revenue_$500,000-$1 MILLION","POS_Revenue_LESS THAN $500,000",POS_Revenue_OVER $1 BILLION,schema_id,correlation,volume_sku
0,317,UNDER 5000SF,1777.219873,0,1,A1,-95.3526,29.76921,ABRAM AYALA,ACE COMFORT AIR COND & HTG,...,0,0,0,0,0,1,0,NaN,NaN,NaN
1,188,0,36231.895740,MULTIFAMILY,1,B1,-95.3539,29.76787,CARLA BISONG,BISONG ART GALLERY,...,0,0,0,0,0,1,0,NaN,NaN,NaN
2,188,0,36231.895740,MULTIFAMILY,1,B1,-95.3539,29.76787,0,IRMA SHIPPING,...,0,0,0,0,0,0,0,NaN,NaN,NaN
3,188,0,36231.895740,MULTIFAMILY,1,B1,-95.3539,29.76787,THEODORE REX,THEODORE REX,...,0,0,0,0,0,1,0,S01,0.25,362.318957
4,188,0,36231.895740,MULTIFAMILY,1,B1,-95.3539,29.76787,THEODORE REX,THEODORE REX,...,0,0,0,0,0,1,0,S02,0.25,362.318957


In [175]:
# Check shape of supply df for feature matrix
supply_dummies.shape

(227, 20)

In [176]:
# Check shape of demand df for feature matrix
combined_demand_df.shape


(6779, 419)

In [177]:
# Grab supply features
supply_features = supply_dummies[['schema_id','avg_stock','avg_price_x','avg_price_y', 'avg_mktshare']]

In [178]:
# Merge demand and supply features on schema_id
mega_df = pd.merge(combined_demand_df, supply_features, on='schema_id')
mega_df.head()

,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClassCd,center_x,center_y,POS_Contact,POS_Name,...,"POS_Revenue_$500,000-$1 MILLION","POS_Revenue_LESS THAN $500,000",POS_Revenue_OVER $1 BILLION,schema_id,correlation,volume_sku,avg_stock,avg_price_x,avg_price_y,avg_mktshare
0,188,0,36231.89574,MULTIFAMILY,1,B1,-95.3539,29.76787,THEODORE REX,THEODORE REX,...,0,1,0,S01,0.25,362.318957,802075,0.22,4.49,176456
1,188,0,36231.89574,MULTIFAMILY,1,B1,-95.3539,29.76787,THEODORE REX,THEODORE REX,...,0,1,0,S01,0.25,362.318957,802075,0.22,4.49,176456
2,188,0,36231.89574,MULTIFAMILY,1,B1,-95.3539,29.76787,THEODORE REX,THEODORE REX,...,0,1,0,S01,0.25,362.318957,802075,0.22,4.49,176456
3,188,0,36231.89574,MULTIFAMILY,1,B1,-95.3539,29.76787,THEODORE REX,THEODORE REX,...,0,1,0,S01,0.25,362.318957,802075,0.22,4.49,176456
4,188,0,36231.89574,MULTIFAMILY,1,B1,-95.3539,29.76787,THEODORE REX,THEODORE REX,...,0,1,0,S01,0.25,362.318957,802075,0.22,4.49,176456


In [180]:
combined_demand_df = pd.get_dummies(combined_demand_df, columns=['schema_id'])

In [181]:
# Grab demand features
demand_features = combined_demand_df[['SqFt_est','correlation','sectorid']]

In [183]:
# Merge mega df and matrix on schema_id
mega_df = pd.merge(mega_df, matrix_data_df, left_on='POS_TYPE_focus', right_on='POS_TYPE_focus', how='left')
mega_df.head()

,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClassCd,center_x,center_y,POS_Contact,POS_Name,...,B02_x,B03_x,Total_x,S01_y,S02_y,S03_y,B01_y,B02_y,B03_y,Total_y
0,188,0,36231.89574,MULTIFAMILY,1,B1,-95.3539,29.76787,THEODORE REX,THEODORE REX,...,0.25,0.25,1.5,0.25,0.25,0.25,0.25,0.25,0.25,1.5
1,188,0,36231.89574,MULTIFAMILY,1,B1,-95.3539,29.76787,THEODORE REX,THEODORE REX,...,0.25,0.25,1.5,0.25,0.25,0.25,0.25,0.25,0.25,1.5
2,188,0,36231.89574,MULTIFAMILY,1,B1,-95.3539,29.76787,THEODORE REX,THEODORE REX,...,0.25,0.25,1.5,0.25,0.25,0.25,0.25,0.25,0.25,1.5
3,188,0,36231.89574,MULTIFAMILY,1,B1,-95.3539,29.76787,THEODORE REX,THEODORE REX,...,0.25,0.25,1.5,0.25,0.25,0.25,0.25,0.25,0.25,1.5
4,188,0,36231.89574,MULTIFAMILY,1,B1,-95.3539,29.76787,THEODORE REX,THEODORE REX,...,0.25,0.25,1.5,0.25,0.25,0.25,0.25,0.25,0.25,1.5


In [184]:
combined_demand_df.fillna(0)
combined_demand_df['volume_sku'].shape

(6779,)

In [185]:
supply_features.fillna(0)
demand_features.fillna(0)

,SqFt_est,correlation,sectorid
0,1777.219873,0.00,1
1,36231.895740,0.00,1
2,36231.895740,0.00,1
3,36231.895740,0.25,1
4,36231.895740,0.25,1
...,...,...,...
6774,64395.469060,0.00,8
6775,64395.469060,0.00,8
6776,64395.469060,0.00,8
6777,64395.469060,0.00,8


## Compile, Train and Evaluate the Model

In [186]:
# define y & X for training
y = combined_demand_df['volume_sku']
X = pd.concat([supply_features, demand_features, combined_demand_df.drop(['POS_TYPE_focus'], axis=1)], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [187]:
# import and start on the Random Forest model
from sklearn.ensemble import RandomForestClassifier

# n estimator - creates a model with 3 trees in the forest
# max depth - each tree will have 3 levels
rfc = RandomForestClassifier(n_estimators=3, random_state=42)

# ************************************************************************ 
Ran into the issue that it's not able convert the string into a float which was tried and attempted in the beginning but it would affect the training(train_test_split). The best way was to compress the files and merging them which is why the 'pri_comp_encoded' file was created to not run into this issue without it altering our data too much for a result that would not work with answering our question about finding the relationship between the supply and demand data 
# ************************************************************************

In [189]:
# use fit RandomForestClassifier
rfc.fit(X_train, y_train)
# Predict the test set labels
y_pred = rfc.predict(X_test)

ValueError: could not convert string to float: 'B01'

The Code below was written prior for the Random Forest Model since I was trying to peace everything together 

In [ ]:
# Visualize each of the trees and how they dividing the data
# Import `tree` module
from sklearn import tree

# The name of each column
features = X.columns.values
# The name of each class 
classes = ['0', '1', '2'] 

for estimator in rfc.estimators_:
    print(estimator)
    plt.figure(figsize=(12,6))
    tree.plot_tree(estimator,
                   feature_names=features,
                   class_names=classes,
                   fontsize=8, 
                   filled=True, 
                   rounded=True)
    plt.show()

In [ ]:
# Receive classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d').set_title('Maternal risks confusion matrix (0 = low risk, 1 = medium risk, 2 = high risk)')

print(classification_report(y_test,y_pred))

In [ ]:
# Calculate Mean Abs. Error, Mean Squared Error, Root Mean Squared Root Error
from sklearn.metrics import mean_absolute_error, mean_squared_error

print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))
